In [1]:
%reload_ext autoreload
%autoreload 3

import mne
import pandas as pd
import numpy as np
from src.preprocessing import annotations as anch
from pathlib import Path
import re

# data directories
data_path = f'/Volumes/Extreme SSD/PhD/MPI-LEMON/EEG_Raw_BIDS_ID/'
mpi_path = f'/Volumes/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/'

### Which data are excluded from MPI dataset for processing?

In [69]:
# get all subject ids in preprocessed folder
sub_ids_pro = []
for subject_path in sorted(Path(mpi_path).glob('sub-*')):
  sub_ids_pro.append(re.search('sub-(.*)_(.*)', subject_path.stem).group(1))
sub_ids_pro = set(sub_ids_pro) # drop duplicates

# get all subject ids in raw folder
sub_ids_raw = []
for subject_path in sorted(Path(data_path).glob('sub-*')):
  sub_ids_raw.append(re.search('sub-(.*)', subject_path.stem).group(1))
sub_ids_raw = set(sub_ids_raw) # drop any possible duplicates

# find any ids in sub_ids_raw that are not in sub_ids_pro
sub_ids_excluded = sorted(list(sub_ids_pro - sub_ids_raw))

## Annotations & Data Segmentation
### Data with different annotation patterns

In [74]:
fnames_annot_misbehave = {}
for subject_path in sorted(Path(data_path).glob('sub-*')):
  sub_id = subject_path.stem
  raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False)

  # change annot names
  desc = anch.change_annot_names(raw.annotations.description.copy())

  # check annot names
  points = anch.annotations_checker(desc)

  # save those with deflection
  if sum(points.values()) != 2:
     fnames_annot_misbehave[sub_id] = points
